In [4]:
#2000 emails sent to email_history took 5 minutes
%load_ext autoreload
%autoreload 2
import logging
import warnings
import os

# Mute pandas UserWarning
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

logging.basicConfig(filename= logpath_creation + '\\Email_Scraper.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)


import pandas as pd
from config import imap_password_customplanet, db_username, db_password
from Scraping_Emails.modules.scrapes import scrape
from Scraping_Emails.modules.db_operations_aws import DatabaseConnector
from MySQL_Scripts.modules.email_failures import  gather_and_send_bad_emails_to_db

#What is the difference between update_bad_emails & gather and send bad emails to db
#Consolidating update_bad_emails to email_faulures


email_address = 'team@customplanet.com'
email_pass = imap_password_customplanet
server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com'
database = 'emailcampaign'
db = 'emailcampaign'


# ----------------------------------Updating Faulty Emails in the DB before Send----------------------------------
#Searches GMAIL on the daily for bad emails that were sent back, and sends data to unsubscribed email table
#Goes through all distinct subject lines in the DB

db_connector_email_history = DatabaseConnector(server, database, db_username, db_password, db, 'email_history')
query = db_connector_email_history.SQL_query('SELECT DISTINCT subject FROM [emailcampaign].[dbo].[email_history]')
    
#Iterate through all unique subject lines to update the db table with all faulty email addresses
for line in query['subject']:
    print('\n')
    print(f'Prior Unique Subject Line: {line}')
    gather_and_send_bad_emails_to_db(line, email_address, email_pass, 'Prior Unique Subject Line')

# ------------------------------------packaging process into final function----------------------------------------

def process(subject_line, email_address, email_pass, start_date):

    #create the msgs via scrape, then iterate through list of messages, and lastly cleanse frame
    email_address = 'team@customplanet.com'
    email_pass = imap_password_customplanet
 
    msgs_inbox = scrape.scrape_msgs_outbox_or_inbox('inbox', subject_line, email_address, email_pass, start_date, 'Current Subject Line')
    inbox = scrape.create_msg_frame(msgs_inbox)
    inbox = scrape.cleanse_frame(inbox, 'inbox')
    inbox.name = 'inbox'

    msgs_outbox = scrape.scrape_msgs_outbox_or_inbox('outbox', subject_line, email_address, email_pass, start_date, 'Current Subject Line')
    outbox = scrape.create_msg_frame(msgs_outbox)
    outbox = scrape.cleanse_frame(outbox, 'outbox')
    outbox.name = 'outbox'
        
    if inbox.empty == True:
        thread = pd.DataFrame()
        pass
    else: #this is a left merge on outbox
        thread = scrape.piece_together(outbox, inbox)
        thread = scrape.assign_sentiment(thread)
       
    # #If message was accidentally triggered more than once
    thread = thread.drop_duplicates(subset = ['subject', 'to'], keep='last')
    thread.reset_index(drop = True, inplace = True)
    outbox = scrape.map_reply_outbox(thread, outbox)

    return(inbox, outbox, thread)


# -------------------------------------------

#Get inbox, outbox, and create thread with given subject line back to given date
subject_line = 'Official MLB Jerseys'
inbox, outbox, thread= process(subject_line, email_address, email_pass, '02/01/2024')


# Instantiate the DatabaseConnector class, send over all emails in outbox variable to email_history table that contain the subject line 
#Optional column identifiers to add in to the DB
outbox['email_campaign_tag'] = 'Majestic Round 2'
outbox['sport'] = 'Baseball'    #frame   #table_name

# ----------Send outbox

if outbox.empty != True:
    db_connector_email_history.send(outbox, 'email_history')
else:
    logging.info('Outbox is an empty frame')
    print('Outbox is an empty frame')

# ----------Send thread

if thread.empty != True:
    db_connector_thread = DatabaseConnector(server, database, db_username, db_password, db, 'thread')
    db_connector_thread.send(thread, 'thread')
    pre_existing_thread_updates = db_connector_thread.update_reply_thread(thread)
else:
    logging.info('Thread is an empty frame')
    print('Thread is an empty frame')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\samuel.taylor\OneDrive - Green Dot Public Schools\Desktop\Git_Directory\CP\CustomPlanet_Work\Email_Scraper\Scraping_Emails\modules\db_operations_aws.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SQL = pd.read_sql_query(query, con=self.conn)




Prior Unique Subject Line: Official MLB Jerseys in Stock!!
Prior Unique Subject Line There are no responses to the specified emails search criteria Official MLB Jerseys in Stock!! in the outbox with a filter date of 02/19/2024
Prior Unique Subject Line There are no responses to the specified emails search criteria Delivery Status Notification in the inbox with a filter date of 02/19/2024
No bad emails to update with Official MLB Jerseys in Stock!!
No emails to update


Prior Unique Subject Line: Official Nike MLB Jerseys In Stock!!
Prior Unique Subject Line There are no responses to the specified emails search criteria Official Nike MLB Jerseys In Stock!! in the outbox with a filter date of 02/19/2024
Prior Unique Subject Line There are no responses to the specified emails search criteria Delivery Status Notification in the inbox with a filter date of 02/19/2024
No bad emails to update with Official Nike MLB Jerseys In Stock!!
No emails to update


Prior Unique Subject Line: Official

In [ ]:
#Figure out how to create a space when de-bugging
# New logging instance
# 20-Feb-24 16:24:16 - There are no responses to the specified emails search criteria Official MLB Little League Nike Jerseys - IN STOCK!! in the outbox with a filter date of 02/19/2024
# 20-Feb-24 16:24:17 - There are no responses to the specified emails search criteria Delivery Status Notification in the inbox with a filter date of 02/19/2024
# 20-Feb-24 16:24:17 - No bad emails to update
# 20-Feb-24 16:24:19 - You have email responses to the specified search criteria Official MLB Jerseys in the inbox with a filter date of 2/01/2023
# 20-Feb-24 16:24:23 - You have email responses to the specified search criteria Official MLB Jerseys in the outbox with a filter date of 2/01/2023



#Within the debug figure out how to log out better. 
#Be able to decipher what is a prior line, versus what is current

# scrape_msgs_outbox_or_inbox #called within this func
#also nested in the one below
# gather_and_send_bad_emails_to_db, still hits the scrape_msgs_outbox_or_inbox func

# To DO

In [ ]:
#GOAL
#All main scripts be moved to the root level, reference a singular config file. 

#Clean up codebase with relative imports. 
#Make Sending and Scraping reference the same code base

#Create PNGs on hoodies images using Selenium. 

#Write directly to MySQL DB

#Refine NLP

#Thirdly, what if the email is not directly replied to. In that case. 
# The scenario would be the following, email campaign sent out to sammytaylor2006@yahoo.com
#The customer indirectly reaches out to CP, and not through the thread. This calls for a seperate table.
# and a seperate scrape 

#Integrate SSIS at some point for views
#Integrating prior orders and linking them in a SQL view.

#Predictive analytics based on prior orders

#Put the PNG on the given sport jersey
#Clean the CSV to iterate properly, this should be done. 
#Singularize the db_operations to reference in both